# Projeto - Natural Language Processing (NLP)

**Integrantes**:
- Antonio Lucas Michelon de Almeida
- Pedro Nery Affonso dos Santos
- Rafael Gordon Paves

In [5]:
import pandas as pd
import re
from dotenv import load_dotenv
import os
from google import genai
import json
import time
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
import numpy as np
import torch
from models import FormatoResposta
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from agents import PlaylistGenerator, Validator
import joblib

load_dotenv()

True

In [3]:
# Captura pedido de música
query = input("O que você gostaria de ouvir hoje? ")

## Playlist gerada a partir de uma única requisição ao Gemini (LLM)

In [ ]:
def monta_playlist(texto):
    """
    Função que realiza uma requisição de montagem de playlist diretamente para o Gemini.
    """
    
    prompt = f"""
    Você é um especialista em música e construção de playlists. 
    Você deve analisar o seguinte pedido de playlist e construir uma seleção de músicas que se encaixem no tema solicitado.
    Refira-se EXCLUSIVAMENTE ao pedido fornecido para criar a playlist.
    A playlist deve ser composta por 10 músicas, com o nome da música, do artista e possível featuring.
    A resposta deve ser uma lista em formato JSON, com os seguintes campos:
    - nome_musica: Nome da música
    - artista: Nome do artista
    - featuring: Nome do artista convidado (caso exista)

    Pedido do usuário: {texto}
    """

    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    resposta = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            'response_schema': list[FormatoResposta],
            'temperature': 1.0
            # 'max_output_tokens': 500,
        }
    )
    return resposta

# Resposta do Gemini
resposta = monta_playlist(query)
# Converte a resposta em um dicionário
resposta_dict = json.loads(resposta.text)
# Cria um DataFrame a partir da lista de dicionários
df = pd.DataFrame(resposta_dict)
# Renomeia as colunas
df.rename(columns={'nome_musica': 'Nome da Música', 'artista': 'Artista', 'featuring': 'Featuring'}, inplace=True)
df

,Nome da Música,Artista,Featuring
0,Umbrella,Rihanna,Jay-Z
1,Crazy in Love,Beyoncé,Jay-Z
2,Family Affair,Mary J. Blige,None
3,Yeah!,Usher,Lil Jon & Ludacris
4,Hot in Herre,Nelly,None
5,Gold Digger,Kanye West,Jamie Foxx
6,Empire State of Mind,Jay-Z,Alicia Keys
7,In Da Club,50 Cent,None
8,Lose Control,Missy Elliott,Ciara & Fatman Scoop
9,My Boo,Usher,Alicia Keys


## Playlist gerada a partir de interações entre agentes

In [ ]:
gerador = PlaylistGenerator()
critico = Validator(query)

# Envia ao gerador de Playlist o request do usuario
playlist = gerador.initialRequest(query).text

repeticoes = 5
for _ in range(repeticoes):
    feedback = critico.refactorRequest(playlist)   
    playlist = gerador.refactorPlaylist(feedback).text

## Playlist gerada a partir de BERT embeddings

In [ ]:
df = pd.read_csv('data/songs_filtrado.csv')
X = df["lyrics"]
y = df['tag']

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

# Utiliza a GPU para fazer as operações, fazendo muito mais rápido para modelos grandes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_embeddings(text, model, tokenizer):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[0, 0, :].cpu()

        # Para nao gastar tanta memoria
        del inputs, outputs
    return cls_embedding

batch_size = 500
for batch_i, inicio in enumerate(tqdm(range(0, len(X), batch_size))):
    # Se chegou ao fim, garante nao passar o index
    fim = inicio+batch_size if inicio+batch_size < len(X) else len(X)

    # Pega pequena amostra, para evitar estourar a memoria RAM (kk)
    batch = X.iloc[inicio:fim]

    embeddings = []
    for i in range(len(batch)):
        e = get_embeddings(batch.iloc[i], model, tokenizer)
        # print(f"{batch.iloc[i]} :{e.detach().numpy()}")
        embeddings.append(e.detach().numpy())
    
    embeddings = np.array(embeddings)

    # Salva os embeddings por batch, depois vamos juntar tudo
    np.save(f'embeddings/bert_emb_{batch_i}.npy', embeddings) 

    # Libera variaveis pesadas
    del embeddings
    # Por usar GPU
    torch.cuda.empty_cache()

100%|██████████| 575/575 [51:22<00:00,  5.36s/it]


In [ ]:
# embeddings = np.array([])

for i in tqdm(range(len(os.listdir("./embeddings")))):
    f = f"./embeddings/bert_emb_{i}.npy"

    if i == 0:
        embeddings = np.load(f)
        continue

    embeddings_load = np.load(f)
    embeddings = np.append(embeddings, embeddings_load, axis=0)

np.save(f'bert_final.npy', embeddings)

del embeddings_load
del embeddings

In [6]:
df = pd.read_csv('songs_filtrado.csv')
y = df['tag']
embeddings = np.load("./bert_final.npy")
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.2, random_state=42)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

c:\Users\PEDRO\OneDrive\Documentos\Insper\2025.1\NLP\playlist-generator-nlp\env\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

     country       0.59      0.31      0.41      1024
        misc       0.83      0.65      0.73      1214
         pop       0.63      0.70      0.66     15513
         rap       0.87      0.93      0.90     27106
          rb       0.49      0.18      0.26      3169
        rock       0.63      0.59      0.61      9427

    accuracy                           0.75     57453
   macro avg       0.68      0.56      0.59     57453
weighted avg       0.74      0.75      0.74     57453



In [7]:
joblib.dump(clf, 'bert_logistic_model.joblib')

['bert_logistic_model.joblib']

In [9]:
# Carregar modelo e dados
clf = joblib.load('bert_logistic_model.joblib')
embeddings = np.load('bert_final.npy')
metadata = pd.read_csv('songs_filtrado.csv')

# Carregar BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained("bert-base-uncased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)

ImportError: 
BertModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [ ]:
def get_embedding(text, model, tokenizer):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[0, 0, :].cpu()
        del inputs, outputs
    return cls_embedding.numpy()

def gerar_playlist_bert(query, top_n=10):
    """
    Gera uma playlist baseada na classificação de estilo usando embeddings BERT
    """
    # Gera embedding da query
    embedding_query = get_embedding(query, bert_model, tokenizer).reshape(1, -1)

    # Classifica a query em uma das tags
    predicted_tag = clf.predict(embedding_query)[0]
    print(f"Tag prevista: {predicted_tag}")

    # Filtra as músicas dessa tag
    mask = metadata['tag'] == predicted_tag
    subset_embeddings = embeddings[mask.values]
    subset_metadata = metadata[mask.values]

    # Calcula similaridade (cosine similarity)
    from sklearn.metrics.pairwise import cosine_similarity
    sims = cosine_similarity(embedding_query, subset_embeddings)[0]

    # Seleciona os top N mais similares
    indices = sims.argsort()[::-1][:top_n]

    playlist = subset_metadata.iloc[indices]
    return playlist[['title', 'artist', 'tag']]

In [ ]:
playlist = gerar_playlist_bert(query)
print(playlist)